# SARAH Toolbox
## Collection of models to be run from a docker image
Launch virtual docker image: 
[![Binder](http://mybinder.org/badge.svg)](http://mybinder.org/repo/restrepo/SimplifiedDM-SDFDM-Toolbox)

[Jupyter home](./) (Files, New -> Terminal, etc)
[![Home](http://www.incredimail.com/images/nav%20bar/home-icon.png)](./)

### Highly recommended:
[Run from a terminal](../terminals/1):
[![T](http://mysitemyway.com/_shared/images/icons/full_color/terminal.png)](../terminals/1)
## Implemented models
Each model is to be run in a specific virtual machine. Follow the binder button in each github repo
* `SARAH/Models/SSDM/` (for test purposes)
* `SARAH/Models/SM/HighScale` (For RGE running: ./butler SM/HighScale) [Repo](https://github.com/restrepo/SM-Toolbox/)
* `SARAH/Models/SimplifiedDM/DFDM` (This image)
* `SARAH/Models/SimplifiedDM/SDFDM` (This image)

## Instructions to compile the model
In `SPHENO` and `micromegas`

See possible analysis based on the models in  [./tests](./tests) folder.

Below we define the model to be compiled:

Choose between
* SimplifiedDMDFDM: Doublet Fermion Dark matter
* SimplifiedDMSDFDM: Singlet Doublet Fermion Dark matter

In [1]:
MODEL=SimplifiedDMSDFDM

To better control in the outputs, it is recomended that the commands be executed [from a terminal](../terminals/1). Only possible errors are to be shown below

In [2]:
./compile_spheno_directly.sh $MODEL > /dev/null

make a work dir

In [3]:
mkdir -p test_compilation
cd test_compilation

In [4]:
LHAINPUT=../SPHENO/$MODEL/LesHouches.in.$MODEL
if [ -f ../SARAH/Models/$MODEL/LesHouches.in.$MODEL ]; then
   LHAINPUT=../SARAH/Models/$MODEL/LesHouches.in.$MODEL
fi

Run SPheno and generate LHA output file

In [5]:
../SPHENO/bin/SPheno$MODEL $LHAINPUT

 Calculating branching ratios and decay widths
 Calculating low energy constraints
 Writing output files
 Finished!


In [6]:
pwd

/home/restrepo/prog/toolbox/toolbox/test_compilation


Compile `micromegas` model

In [7]:
cd ../micromegas
make > /dev/null
make > /dev/null
cd $MODEL
make main=CalcOmega.cpp > /dev/null

### Include Indirect Detection

In [15]:
micromegas_with_DDetection=CalcOmega_with_DDetection_MOv4.2.cpp 
IDmsg="======== Indirect Detection ========"
if [ -f "$micromegas_with_DDetection" ]  && [ ! "$(grep "$IDmsg" "$micromegas_with_DDetection" )" ]; then
    tmpfile=$(mktemp /tmp/CalcOmega_UP.XXXXXX)
    grep -B 1000 'fclose(channels);' "$micromegas_with_DDetection" | grep -v 'fclose(channels);' > "$tmpfile"
    echo "
{
  double Emin=1,/* Energy cut  in GeV   */  sigmaV;
  double vcs_gz,vcs_gg;
  char txt[100];
  double SpA[NZ],SpE[NZ],SpP[NZ];
  double FluxA[NZ],FluxE[NZ],FluxP[NZ];
  double * SpNe=NULL,*SpNm=NULL,*SpNl=NULL;
  double Etest=Mcdm/2;

  printf(\"\n$IDmsg\n\"); 
  sigmaV=calcSpectrum(1+2+4,SpA,SpE,SpP,SpNe,SpNm,SpNl ,&err);

}" >> "$tmpfile"

  grep -A 1000 'fclose(channels);' "$micromegas_with_DDetection" >> "$tmpfile"
  mv "$tmpfile" "$micromegas_with_DDetection"
else 
  echo "$IDmsg"
  echo "already included"
fi

======== Indirect Detection ========
already included


In [10]:
make main="$micromegas_with_DDetection" > /dev/null
cd ../IDM
make main=main.c > /dev/null

Run micromegas

In [12]:
cd ../../test_compilation
../micromegas/$MODEL/$(basename "$micromegas_with_DDetection" .cpp) SPheno.spc.$MODEL 

bash: cd: ../../test_compilation: No such file or directory

Masses of odd sector Particles:
~Chi : MChi1 =   199.7 || ~Chi : MChi2 =   200.0 || ~Re  : MRe   =   200.0 
~Chi : MChi3 =  2000.3 || 
Xf=2.75e+01 Omega h^2=1.66e-02

# Channels which contribute to 1/(omega) more than 1%.
# Relative contributions in % are displayed
   13% ~Chi1 ~Re ->A Wm 
   10% ~Chi1 ~Re ->Z Wm 
    9% ~Re ~re ->u1 U1 
    9% ~Re ~re ->u2 U2 
    7% ~Chi1 ~Chi1 ->Wp Wm 
    6% ~Re ~re ->d1 D1 
    6% ~Re ~re ->d2 D2 
    6% ~Re ~re ->d3 D3 
    5% ~Re ~re ->u3 U3 
    4% ~Re ~re ->e1 E1 
    4% ~Re ~re ->e2 E2 
    4% ~Re ~re ->e3 E3 
    4% ~Re ~Re ->Wm Wm 
    3% ~Re ~re ->A A 
    3% ~Re ~re ->A Z 
    2% ~Chi1 ~Re ->d3 U3 
    1% ~Re ~re ->nu1 Nu1 
    1% ~Re ~re ->nu2 Nu2 
    1% ~Re ~re ->nu3 Nu3 

==== Calculation of CDM-nucleons amplitudes  =====
         TREE LEVEL
CDM-nucleon micrOMEGAs amplitudes:
proton:  SI  -7.775E-11  SD  0.000E+00
neutron: SI  -7.882E-11  SD  0.000E+00
         BOX DIAGRAMS


## Go now to: 
* [main ipyrhon file](./tests/SimplifiedDM-DFDM.ipynb) → SimplifiedDMDFDM: Doublet Fermion Dark matter
* [main ipyrhon file](./tests/SimplifiedDM-SDFDM.ipynb) → SimplifiedDMSDFDM: Singlet Doublet Fermion Dark matter
